Mount Google Drive

In [2]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/DS5230_Project/

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1VDNu1UgCrikOKRQws_HNNIKQJ4YbCTvu/DS5230_Project


Import Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.cluster import DBSCAN

Load Crime Data

In [4]:
crime_data = pickle.load(open('Code/Data/crime.pkl', 'rb'))

In [5]:
crime_data.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I192068249,2647,Other,THREATS TO DO BODILY HARM,B2,280,None,2015-08-28 10:20:00,2015,8,Friday,10,Part Two,WASHINGTON ST,42.330119,-71.084251,"(42.33011862, -71.08425106)"
1,I192061894,1106,Confidence Games,FRAUD - CREDIT CARD / ATM FRAUD,C11,356,None,2015-08-20 00:00:00,2015,8,Thursday,0,Part Two,CHARLES ST,42.300605,-71.061268,"(42.30060543, -71.06126785)"
2,I192038828,1107,Fraud,FRAUD - IMPERSONATION,A1,172,None,2015-11-02 12:24:00,2015,11,Monday,12,Part Two,ALBANY ST,42.334288,-71.072395,"(42.33428841, -71.07239518)"
3,I192008877,1107,Fraud,FRAUD - IMPERSONATION,E18,525,None,2015-07-31 10:00:00,2015,7,Friday,10,Part Two,WINGATE RD,42.237009,-71.129566,"(42.23700950, -71.12956606)"
4,I182090828,1102,Fraud,FRAUD - FALSE PRETENSE / SCHEME,D4,159,None,2015-12-01 12:00:00,2015,12,Tuesday,12,Part Two,UPTON ST,42.342432,-71.072258,"(42.34243222, -71.07225766)"


Fill missing Offense Code Groups based on Offense Codes

In [6]:
offense_code_group = pd.read_csv('Code/Data/Offense Codes.csv')

In [7]:
offense_code_group

,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_CODE_GROUP_SUMMARIZED
0,413,AGGRAVATED ASSAULT,ASSAULT
1,423,AGGRAVATED ASSAULT,ASSAULT
2,432,AGGRAVATED ASSAULT,ASSAULT
3,403,AGGRAVATED ASSAULT,ASSAULT
4,404,AGGRAVATED ASSAULT,ASSAULT
...,...,...,...
214,2905,VIOLATIONS,VIOLATIONS
215,2907,VIOLATIONS,VIOLATIONS
216,2906,VIOLATIONS,VIOLATIONS
217,2910,VIOLATIONS,VIOLATIONS


In [8]:
crime_data_cleaned = pd.merge(crime_data.drop('OFFENSE_CODE_GROUP', axis = 1), offense_code_group, on = 'OFFENSE_CODE', how = 'left')

Load Census Data

In [9]:
census_data = pd.read_csv('Code/Data/Boston Neighborhood Census Data 2015-2020.csv')

Fill missing values with prior year since that will be the most similar

In [10]:
census_data = census_data.sort_values(by = ['Neighborhood', 'Year'], ascending = True)
census_data = census_data.fillna(method = 'ffill')

In [11]:
census_data.to_csv('Code/Data/census_data.csv', index = False)

In [12]:
census_data.head()

,Year,Neighborhood,Population,Median Age,White Alone,Black / African American,Hispanic,Asian Alone,Other Races,Less than High School,High School Graduate,Some College,Bachelor's Degree or more,Poverty rate
0,2015,Allston,19761,27.0,0.573,0.058,0.161,0.168,0.039,0.087,0.132,0.138,0.643,0.313
23,2016,Allston,19399,25.0,0.561,0.056,0.124,0.215,0.044,0.077,0.109,0.135,0.679,0.324
46,2017,Allston,19363,26.0,0.542,0.062,0.138,0.217,0.041,0.068,0.093,0.120,0.719,0.311
69,2018,Allston,19627,26.0,0.530,0.048,0.138,0.244,0.040,0.054,0.078,0.119,0.748,0.302
92,2019,Allston,19261,27.5,0.511,0.057,0.134,0.250,0.048,0.055,0.074,0.119,0.752,0.279


In [13]:
census_data.describe()

,Year,Population,Median Age,White Alone,Black / African American,Hispanic,Asian Alone,Other Races,Less than High School,High School Graduate,Some College,Bachelor's Degree or more,Poverty rate
count,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000
mean,2017.489051,29317.481752,33.203650,0.547044,0.155803,0.155285,0.105562,0.036314,0.110253,0.159635,0.156679,0.570511,0.210898
std,1.715487,25375.255400,5.596341,0.227520,0.189102,0.112361,0.087637,0.020450,0.080982,0.094252,0.077852,0.230016,0.142754
min,2015.000000,322.000000,20.000000,0.060000,0.003000,0.009000,0.000000,0.014000,0.000000,0.017000,0.049000,0.045000,0.014000
25%,2016.000000,10805.000000,31.000000,0.412000,0.041000,0.089000,0.038000,0.023000,0.055000,0.088000,0.103000,0.421000,0.119000
50%,2017.000000,26170.000000,33.000000,0.564000,0.062000,0.120000,0.086000,0.033000,0.087000,0.133000,0.130000,0.635000,0.180000
75%,2019.000000,37094.000000,35.700000,0.718000,0.164000,0.200000,0.159000,0.041000,0.156000,0.215000,0.207000,0.745000,0.236000
max,2020.000000,126909.000000,52.000000,0.908000,0.744000,0.579000,0.599000,0.143000,0.401000,0.372000,0.410000,0.912000,0.829000


Add District

In [14]:
neighborhood_districts = pd.read_csv('Code/Data/Boston Neighborhood Districts.csv')

In [15]:
neighborhood_districts.head()

,District,Neighborhood
0,A1,Downtown
1,A15,Charlestown
2,A7,East Boston
3,B2,Roxbury
4,B3,Mattapan


In [16]:
district_population = pd.merge(census_data, neighborhood_districts, on = 'Neighborhood')[['Year', 'District', 'Population']].groupby(['Year', 'District']).sum().reset_index()
district_population = district_population.rename(columns = {'Population' : 'Total Population'})

In [17]:
district_population.head()

,Year,District,Total Population
0,2015,A1,41260
1,2015,A15,18058
2,2015,A7,44989
3,2015,B2,73185
4,2015,B3,24268


In [18]:
district_census_data = pd.merge(census_data, neighborhood_districts, on = 'Neighborhood')
district_census_data = pd.merge(district_census_data, district_population, on = ['Year', 'District'])

district_census_data['Percent of District'] = district_census_data['Population'] / district_census_data['Total Population']
district_census_data = district_census_data.drop('Total Population', axis = 1)

district_census_data['Median Age'] = district_census_data['Median Age'] * district_census_data['Percent of District']
district_census_data['White Alone'] = district_census_data['White Alone'] * district_census_data['Percent of District']
district_census_data['Black / African American'] = district_census_data['Black / African American'] * district_census_data['Percent of District']
district_census_data['Hispanic'] = district_census_data['Hispanic'] * district_census_data['Percent of District']
district_census_data['Asian Alone'] = district_census_data['Asian Alone'] * district_census_data['Percent of District']
district_census_data['Other Races'] = district_census_data['Other Races'] * district_census_data['Percent of District']
district_census_data['Less than High School'] = district_census_data['Less than High School'] * district_census_data['Percent of District']
district_census_data['High School Graduate'] = district_census_data['High School Graduate'] * district_census_data['Percent of District']
district_census_data['Some College'] = district_census_data['Some College'] * district_census_data['Percent of District']
district_census_data["Bachelor's Degree or more"] = district_census_data["Bachelor's Degree or more"] * district_census_data['Percent of District']
district_census_data['Poverty rate'] = district_census_data['Poverty rate'] * district_census_data['Percent of District']

district_census_data = district_census_data.groupby(['Year', 'District']).sum().reset_index()
district_census_data = district_census_data.drop('Percent of District', axis = 1)

district_census_data = district_census_data.rename(columns = {'Year' : 'YEAR', 'District' : 'DISTRICT'})

In [19]:
district_census_data.head()

,YEAR,DISTRICT,Population,Median Age,White Alone,Black / African American,Hispanic,Asian Alone,Other Races,Less than High School,High School Graduate,Some College,Bachelor's Degree or more,Poverty rate
0,2015,A1,41260,32.661270,0.714909,0.027158,0.060292,0.172636,0.024780,0.106509,0.076753,0.094916,0.721932,0.165535
1,2015,A15,18058,34.000000,0.696000,0.086000,0.110000,0.087000,0.020000,0.089900,0.159000,0.131000,0.620000,0.203000
2,2015,A7,44989,34.000000,0.320000,0.024000,0.579000,0.036000,0.041000,0.321500,0.310000,0.164000,0.205000,0.203000
3,2015,B2,73185,30.056774,0.233795,0.412733,0.250983,0.068226,0.034263,0.217134,0.258737,0.222831,0.301521,0.372753
4,2015,B3,24268,38.000000,0.061000,0.737000,0.159000,0.023000,0.019000,0.212800,0.337000,0.290000,0.160000,0.221000


In [20]:
district_census_data.to_csv('Code/Data/district_census_data.csv', index = False)

Join Crime and Census Data

In [21]:
crime_census_data = pd.merge(crime_data_cleaned, district_census_data, on = ['YEAR', 'DISTRICT'])

In [22]:
crime_census_data.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,...,White Alone,Black / African American,Hispanic,Asian Alone,Other Races,Less than High School,High School Graduate,Some College,Bachelor's Degree or more,Poverty rate
0,I192068249,2647,THREATS TO DO BODILY HARM,B2,280,None,2015-08-28 10:20:00,2015,8,Friday,...,0.233795,0.412733,0.250983,0.068226,0.034263,0.217134,0.258737,0.222831,0.301521,0.372753
1,I182074094,2629,HARASSMENT,B2,258,None,2015-09-14 09:31:00,2015,9,Monday,...,0.233795,0.412733,0.250983,0.068226,0.034263,0.217134,0.258737,0.222831,0.301521,0.372753
2,I182054888,2670,CRIMINAL HARASSMENT,B2,326,None,2015-07-12 15:37:00,2015,7,Sunday,...,0.233795,0.412733,0.250983,0.068226,0.034263,0.217134,0.258737,0.222831,0.301521,0.372753
3,I182054888,3170,INTIMIDATING WITNESS,B2,326,None,2015-07-12 15:37:00,2015,7,Sunday,...,0.233795,0.412733,0.250983,0.068226,0.034263,0.217134,0.258737,0.222831,0.301521,0.372753
4,I182054888,2647,THREATS TO DO BODILY HARM,B2,326,None,2015-07-12 15:37:00,2015,7,Sunday,...,0.233795,0.412733,0.250983,0.068226,0.034263,0.217134,0.258737,0.222831,0.301521,0.372753


In [23]:
crime_census_data.to_pickle('Code/Data/crime_census.pkl')

Create dataset for model

In [24]:
crime_census_data = pickle.load(open('Code/Data/crime_census.pkl', 'rb'))

In [25]:
crime_census_data['COUNT'] = 1

In [26]:
crime_census_data.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,...,Black / African American,Hispanic,Asian Alone,Other Races,Less than High School,High School Graduate,Some College,Bachelor's Degree or more,Poverty rate,COUNT
0,I192068249,2647,THREATS TO DO BODILY HARM,B2,280,None,2015-08-28 10:20:00,2015,8,Friday,...,0.412733,0.250983,0.068226,0.034263,0.217134,0.258737,0.222831,0.301521,0.372753,1
1,I182074094,2629,HARASSMENT,B2,258,None,2015-09-14 09:31:00,2015,9,Monday,...,0.412733,0.250983,0.068226,0.034263,0.217134,0.258737,0.222831,0.301521,0.372753,1
2,I182054888,2670,CRIMINAL HARASSMENT,B2,326,None,2015-07-12 15:37:00,2015,7,Sunday,...,0.412733,0.250983,0.068226,0.034263,0.217134,0.258737,0.222831,0.301521,0.372753,1
3,I182054888,3170,INTIMIDATING WITNESS,B2,326,None,2015-07-12 15:37:00,2015,7,Sunday,...,0.412733,0.250983,0.068226,0.034263,0.217134,0.258737,0.222831,0.301521,0.372753,1
4,I182054888,2647,THREATS TO DO BODILY HARM,B2,326,None,2015-07-12 15:37:00,2015,7,Sunday,...,0.412733,0.250983,0.068226,0.034263,0.217134,0.258737,0.222831,0.301521,0.372753,1


In [27]:
crime_counts = crime_census_data[['DISTRICT', 'OFFENSE_CODE_GROUP', 'YEAR', 'COUNT', 'Population']].groupby(['DISTRICT', 'OFFENSE_CODE_GROUP', 'YEAR', 'Population']).sum().reset_index()
crime_counts['COUNT'] = crime_counts['COUNT'] / crime_counts['Population'] * 1000
crime_counts = crime_counts.pivot(index = ['DISTRICT', 'YEAR', 'Population'], columns = 'OFFENSE_CODE_GROUP', values = 'COUNT').reset_index().fillna(0).drop('Population', axis = 1)

In [28]:
crime_counts.head()

OFFENSE_CODE_GROUP,DISTRICT,YEAR,AGGRAVATED ASSAULT,AIRCRAFT,ARSON,ASSEMBLY OR GATHERING VIOLATIONS,AUTO THEFT,AUTO THEFT RECOVERY,BALLISTICS,BIOLOGICAL THREAT,...,RESTRAINING ORDER VIOLATIONS,ROBBERY,SEARCH WARRANTS,SERVICE,SIMPLE ASSAULT,TOWED,VANDALISM,VERBAL DISPUTES,VIOLATIONS,WARRANT ARRESTS
0,A1,2015,3.635482,0.0,0.072710,1.187591,1.720795,0.218129,0.024237,0.000000,...,0.266602,2.593311,0.387785,0.048473,9.961222,4.726127,5.307804,0.557441,1.623849,5.307804
1,A1,2016,5.777461,0.0,0.071327,1.212553,2.044698,0.404184,0.190204,0.000000,...,0.689491,3.851641,0.261531,0.047551,13.647171,8.773181,8.464099,1.307656,1.688065,11.222064
2,A1,2017,5.680756,0.0,0.023378,1.379278,2.501403,0.561062,0.093510,0.046755,...,0.841593,4.161212,0.397419,0.116888,13.582383,9.351038,7.457453,2.010473,1.098747,12.647279
3,A1,2018,5.770646,0.0,0.000000,1.195512,1.793268,0.275887,0.091962,0.000000,...,0.689719,3.678499,0.252897,0.091962,16.392312,8.345595,7.816811,1.517381,1.471400,7.586905
4,A1,2019,5.909238,0.0,0.023083,0.900235,2.308296,0.300078,0.138498,0.023083,...,0.000000,3.000785,0.230830,0.092332,17.173722,6.855639,6.486312,2.585292,2.262130,3.323946


In [29]:
crime_counts2 = crime_census_data[['DISTRICT', 'OFFENSE_CODE_GROUP_SUMMARIZED', 'YEAR', 'COUNT', 'Population']].groupby(['DISTRICT', 'OFFENSE_CODE_GROUP_SUMMARIZED', 'YEAR', 'Population']).sum().reset_index()
crime_counts2['COUNT'] = crime_counts2['COUNT'] / crime_counts2['Population'] * 1000
crime_counts2 = crime_counts2.pivot(index = ['DISTRICT', 'YEAR', 'Population'], columns = 'OFFENSE_CODE_GROUP_SUMMARIZED', values = 'COUNT').reset_index().fillna(0).drop('Population', axis = 1)

In [30]:
crime_counts2.head()

OFFENSE_CODE_GROUP_SUMMARIZED,DISTRICT,YEAR,AIRCRAFT,ARSON,ASSAULT,ASSEMBLY OR GATHERING VIOLATIONS,AUTO THEFT,BALLISTICS,BIOLOGICAL THREAT,BOMB HOAX,...,PROSTITUTION,RESTRAINING ORDER VIOLATIONS,ROBBERY,SEARCH WARRANTS,SERVICE,TOWED,VANDALISM,VERBAL DISPUTES,VIOLATIONS,WARRANT ARRESTS
0,A1,2015,0.0,0.072710,13.596704,1.187591,1.938924,0.024237,0.000000,0.048473,...,0.169656,0.266602,2.593311,0.387785,0.048473,4.726127,5.307804,0.557441,1.623849,5.307804
1,A1,2016,0.0,0.071327,19.424631,1.212553,2.448883,0.190204,0.000000,0.142653,...,0.309082,0.689491,3.851641,0.261531,0.047551,8.773181,8.464099,1.307656,1.688065,11.222064
2,A1,2017,0.0,0.023378,19.263138,1.379278,3.062465,0.093510,0.046755,0.070133,...,0.724705,0.841593,4.161212,0.397419,0.116888,9.351038,7.457453,2.010473,1.098747,12.647279
3,A1,2018,0.0,0.000000,22.162958,1.195512,2.069156,0.091962,0.000000,0.275887,...,0.000000,0.689719,3.678499,0.252897,0.091962,8.345595,7.816811,1.517381,1.471400,7.586905
4,A1,2019,0.0,0.023083,23.082960,0.900235,2.608374,0.138498,0.023083,0.046166,...,0.023083,0.000000,3.000785,0.230830,0.092332,6.855639,6.486312,2.585292,2.262130,3.323946


In [31]:
model_data_yearly = crime_census_data[['DISTRICT', 'YEAR', 'MONTH', 'DAY_OF_WEEK', 'HOUR', 'Median Age', 'White Alone', 'Black / African American', 'Hispanic', 'Asian Alone', 'Other Races', 'Less than High School', 'High School Graduate', 'Some College', "Bachelor's Degree or more", 'Poverty rate']]

In [32]:
model_data_yearly = model_data_yearly.replace({'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7})

In [33]:
model_data_yearly = model_data_yearly.groupby(['DISTRICT', 'YEAR']).mean().reset_index()

In [34]:
model_data_yearly1 = pd.merge(model_data_yearly, crime_counts, on = ['YEAR', 'DISTRICT'])

In [35]:
model_data_yearly2 = pd.merge(model_data_yearly, crime_counts2, on = ['YEAR', 'DISTRICT'])

In [36]:
model_data_yearly2

,DISTRICT,YEAR,MONTH,DAY_OF_WEEK,HOUR,Median Age,White Alone,Black / African American,Hispanic,Asian Alone,...,PROSTITUTION,RESTRAINING ORDER VIOLATIONS,ROBBERY,SEARCH WARRANTS,SERVICE,TOWED,VANDALISM,VERBAL DISPUTES,VIOLATIONS,WARRANT ARRESTS
0,A1,2015,9.185894,3.997758,12.467150,32.661270,0.714909,0.027158,0.060292,0.172636,...,0.169656,0.266602,2.593311,0.387785,0.048473,4.726127,5.307804,0.557441,1.623849,5.307804
1,A1,2016,6.546044,4.044214,12.437755,32.115835,0.712439,0.026454,0.060894,0.176923,...,0.309082,0.689491,3.851641,0.261531,0.047551,8.773181,8.464099,1.307656,1.688065,11.222064
2,A1,2017,6.707128,4.110092,12.365662,32.266154,0.703850,0.028123,0.072998,0.172592,...,0.724705,0.841593,4.161212,0.397419,0.116888,9.351038,7.457453,2.010473,1.098747,12.647279
3,A1,2018,6.546260,4.057284,12.550072,33.435971,0.691667,0.029577,0.076794,0.173991,...,0.000000,0.689719,3.678499,0.252897,0.091962,8.345595,7.816811,1.517381,1.471400,7.586905
4,A1,2019,6.734545,4.036364,12.659144,33.560941,0.701228,0.031321,0.072256,0.167790,...,0.023083,0.000000,3.000785,0.230830,0.092332,6.855639,6.486312,2.585292,2.262130,3.323946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,E5,2016,6.491969,3.895478,13.119842,40.122308,0.621829,0.149984,0.157372,0.045059,...,0.000000,0.258102,0.467810,0.290365,0.129051,1.064671,3.823136,2.435837,1.177590,0.854963
68,E5,2017,6.678527,3.988474,13.039840,41.149645,0.610271,0.150721,0.161674,0.050095,...,0.000000,0.174227,0.475165,0.285099,0.110872,1.251267,2.993538,2.787633,0.807780,0.807780
69,E5,2018,6.398635,3.883215,12.967897,41.146840,0.599071,0.154430,0.164810,0.052446,...,0.015661,0.219257,0.438514,0.187935,0.046984,2.239554,2.537117,2.959970,0.610787,0.673432
70,E5,2019,6.617439,3.888828,13.349319,41.382734,0.596050,0.155542,0.166194,0.051379,...,0.000000,0.000000,0.409146,0.157364,0.078682,1.274647,2.218830,2.753867,0.834028,0.094418


In [37]:
#model_data_yearly1.to_pickle('Code/Data/model_data_yearly1.pkl')
#model_data_yearly2.to_pickle('Code/Data/model_data_yearly2.pkl')

model_data_yearly1.to_csv('Code/Data/model_data_yearly1.csv', index = False)
model_data_yearly2.to_csv('Code/Data/model_data_yearly2.csv', index = False)

In [38]:
model_data1 = model_data_yearly1.groupby('DISTRICT').mean().reset_index().drop('YEAR', axis = 1)
model_data2 = model_data_yearly2.groupby('DISTRICT').mean().reset_index().drop('YEAR', axis = 1)

In [39]:
model_data2

,DISTRICT,MONTH,DAY_OF_WEEK,HOUR,Median Age,White Alone,Black / African American,Hispanic,Asian Alone,Other Races,...,PROSTITUTION,RESTRAINING ORDER VIOLATIONS,ROBBERY,SEARCH WARRANTS,SERVICE,TOWED,VANDALISM,VERBAL DISPUTES,VIOLATIONS,WARRANT ARRESTS
0,A1,7.033408,4.034134,12.513945,32.986141,0.700316,0.029173,0.067944,0.175212,0.027454,...,0.207862,0.438652,3.255942,0.261958,0.079964,7.239948,7.067827,1.594640,1.818416,6.784553
1,A15,6.990224,3.826041,13.093500,34.900000,0.719667,0.063833,0.113500,0.080333,0.022833,...,0.000000,0.354359,0.944578,0.253053,0.166541,4.608553,5.037885,2.570483,1.191922,1.192759
2,A7,6.967226,3.962442,13.076501,32.866667,0.333833,0.026833,0.560500,0.041167,0.038167,...,0.003705,0.332539,1.238325,0.122911,0.137881,3.329743,4.537581,2.676015,1.435600,1.296317
3,B2,6.929485,3.913553,13.237799,30.097267,0.224368,0.389967,0.263748,0.079663,0.042449,...,0.100164,0.805532,2.378504,0.702133,0.145011,3.417453,8.010383,9.089829,4.043674,3.692102
4,B3,6.984170,3.949746,13.275074,37.066667,0.064500,0.723833,0.157167,0.019500,0.034833,...,0.019438,2.360219,4.850677,1.769451,0.396645,7.647783,19.247567,26.807097,8.484071,4.878210
5,C11,6.954368,3.935199,13.067816,33.133333,0.219833,0.430833,0.185167,0.101000,0.063000,...,0.089078,0.536698,1.259134,0.223159,0.098609,2.583718,4.759928,4.975176,1.687260,1.881792
6,C6,6.992721,3.969458,12.768413,32.153269,0.773607,0.052529,0.099930,0.052459,0.021453,...,0.078554,0.373794,1.591727,0.248840,0.139623,8.755471,7.343611,3.470135,1.363744,3.342219
7,D14,7.051409,3.976169,12.820425,28.934509,0.620495,0.046337,0.117377,0.174762,0.040817,...,0.109310,0.197901,0.561975,0.194145,0.093932,5.144274,3.598129,1.760091,0.917704,0.656967
8,D4,7.006984,3.955507,12.983868,30.529933,0.607478,0.073335,0.115707,0.167306,0.036406,...,0.046381,0.317460,1.705076,0.094601,0.079262,5.649420,5.130293,2.166338,1.369631,2.904208
9,E13,6.970813,3.879755,13.350468,34.100000,0.547167,0.119000,0.235500,0.058333,0.038500,...,0.004254,0.306405,1.441597,0.318479,0.119816,5.239595,4.924821,3.472188,2.498392,2.159070


In [40]:
#model_data1.to_pickle('Code/Data/model_data1.pkl')
#model_data2.to_pickle('Code/Data/model_data2.pkl')

model_data1.to_csv('Code/Data/model_data1.csv', index = False)
model_data2.to_csv('Code/Data/model_data2.csv', index = False)